<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/notebooks/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

https://web.stanford.edu/class/ee364b/lectures/primal_dual_subgrad_slides.pdf

https://www.cvxpy.org/examples/basic/quadratic_program.html

In [2]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [12]:
# def gradient_maker(grads):
#     from scipy.special import expit
#     sigmoid = lambda x : expit(x)

#     eps = 1e-5
#     lr = 0.01
#     n, d = grads.shape
    
#     g = np.random.randn(d)

#     pbar = tqdm(range(1000))
#     for i in pbar:
#         a = sigmoid(grads@g-eps)
#         g += lr * grads.T@((a-a**2)/np.sum(a))

#         pbar.set_description("Processing %.2f" % (np.sum(grads@g>=0)/n))
#     return g

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class GradientMaker(nn.Module):
    def __init__(self, W, eps=1e-5):
        super().__init__()
        self._eps = eps
        self._n, self._d = W.shape
        self._W = W

        # define parameters
        self._g = nn.Parameter(torch.ones(self._d))
    
    def forward(self):
        z = self._W@self._g-self._eps
        # a = torch.sigmoid(z) * torch.heaviside(z, torch.tensor([1.]))
        a = torch.sigmoid(z) * (z>=0)
        return torch.sum(a)/self._n

shape = (2000, 7840)
grads = 1e-10*(1000*(torch.randn(*shape)+1))*torch.randn(*shape)

gmaker = GradientMaker(grads)
opt = optim.Adam(gmaker.parameters())

get_params = lambda: next(iter(gmaker.parameters())).clone().detach()


gmaker.train()
# pbar = tqdm(range(20))
for epoch in range(200):
    opt.zero_grad()
    y = gmaker()
    loss = -1. * torch.log(y)
    loss.backward()
    opt.step()

    print(f"Epoch:{epoch}, Loss:{loss.item():.3}, PosPortion: {(torch.sum(grads@get_params()>=0)/shape[0]):.3}")

Epoch:0, Loss:2.29, PosPortion: 0.493
Epoch:1, Loss:2.29, PosPortion: 0.493
Epoch:2, Loss:2.29, PosPortion: 0.493
Epoch:3, Loss:2.28, PosPortion: 0.493
Epoch:4, Loss:2.28, PosPortion: 0.493
Epoch:5, Loss:2.28, PosPortion: 0.493
Epoch:6, Loss:2.28, PosPortion: 0.493
Epoch:7, Loss:2.28, PosPortion: 0.493
Epoch:8, Loss:2.28, PosPortion: 0.493
Epoch:9, Loss:2.28, PosPortion: 0.493
Epoch:10, Loss:2.28, PosPortion: 0.493
Epoch:11, Loss:2.28, PosPortion: 0.493
Epoch:12, Loss:2.28, PosPortion: 0.493
Epoch:13, Loss:2.28, PosPortion: 0.493
Epoch:14, Loss:2.28, PosPortion: 0.493
Epoch:15, Loss:2.28, PosPortion: 0.493
Epoch:16, Loss:2.28, PosPortion: 0.493
Epoch:17, Loss:2.28, PosPortion: 0.493
Epoch:18, Loss:2.28, PosPortion: 0.493
Epoch:19, Loss:2.28, PosPortion: 0.493
Epoch:20, Loss:2.28, PosPortion: 0.493
Epoch:21, Loss:2.28, PosPortion: 0.493
Epoch:22, Loss:2.28, PosPortion: 0.493
Epoch:23, Loss:2.28, PosPortion: 0.494
Epoch:24, Loss:2.28, PosPortion: 0.494
Epoch:25, Loss:2.28, PosPortion: 0.

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [4]:
shape = (2000, 7840)
grads = 1e-10*(1000*(np.random.randn(*shape)+1))*np.random.randn(*shape)

In [13]:
g = gradient_maker(grads)

Processing 0.50:  31%|███       | 306/1000 [00:18<00:42, 16.41it/s]


KeyboardInterrupt: ignored

In [6]:
np.min(grads@g)

-4.236641056337677e-05

In [ ]:
import torch